In [1]:
import numpy as np
import torch
import os
import time
import pickle
from tqdm import tqdm 

from transformers import HubertModel, Wav2Vec2FeatureExtractor
from python_speech_features import mfcc, fbank, logfbank, ssc
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import soundfile as sf

import matplotlib.pyplot as plt

In [10]:
## CHANGE this line
#model_path='../First experiments on speech/hubert-base-960'
model_path='D:/! Models/HuBERT-large'

### Directory for saving .npy files with features 
SAVE_PATH = 'embs'


# Wav2Vec2 with default parameters 
extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=False, return_attention_mask=True)
### Loading teh model
model = HubertModel.from_pretrained(model_path, output_attentions=True, output_hidden_states=True)

Some weights of the model checkpoint at D:/! Models/HuBERT-large were not used when initializing HubertModel: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# Faster than computing H0_s via Giotto-TDA 
def prim_tree(adj_matrix):
    infty = torch.max(adj_matrix) + 10
    
    dst = torch.ones(adj_matrix.shape[0]) * infty
    visited = torch.zeros(adj_matrix.shape[0], dtype=bool)
    ancestor = -torch.ones(adj_matrix.shape[0], dtype=int)

    v, s = 0, torch.tensor(0.0)
    for i in range(adj_matrix.shape[0] - 1):
        visited[v] = 1
        ancestor[dst > adj_matrix[v]] = v
        dst = np.minimum(dst, adj_matrix[v])
        dst[visited] = infty
        
        v = np.argmin(dst)
        
        s += adj_matrix[v][ancestor[v]]
    return s.item()

In [11]:
PATH = 'D:/Datasets/CREMA-D/AudioWAV' ## PAth to dataset files

embs = []
embs_zero = []

### Изменить в зависимости от структуры датасета. Приведено для CREMA-D
labels = []
text_to_label = {'ANG' : 0, 'HAP' : 1, 'DIS' : 2, 'FEA' : 3, 'NEU' : 4, 'SAD' : 5}

for filename in tqdm(os.listdir(PATH)):
    code = filename.split('_')[-2]
    labels.append(text_to_label[code])
    
    data, samplerate = sf.read(PATH + '/' + filename)
### ----------------------------------------------------------------------
### Baseline - concatenated embeddings from all layers
  
    inputs = extractor(data, sampling_rate=samplerate, return_tensors="pt").input_values
    with torch.no_grad():
        hidden = model(inputs)["hidden_states"]
        hidden = np.asarray([layer.detach().numpy() for layer in hidden], dtype=np.float32)
        embeddings_mean = np.mean(hidden[:, 0, :, :], 1)
        embeddings_zero = hidden[:, 0, 0, :]
    
    embeddings_mean = embeddings_mean.reshape(embeddings_mean.shape[0] * embeddings_mean.shape[1])
    embeddings_zero = embeddings_zero.reshape(embeddings_zero.shape[0] * embeddings_zero.shape[1])
    
    embs.append(embeddings_mean)
    embs_zero.append(embeddings_zero)

labels = np.array(labels)
for template in [embs, embs_zero]:
    template = np.vstack(template)

100%|████████████████████████████████████████████████████████████████████████████| 7442/7442 [2:18:27<00:00,  1.12s/it]


In [12]:
np.save(SAVE_PATH + '/labels', labels)
np.save(SAVE_PATH + '/X_allembs_large', embs)
np.save(SAVE_PATH + '/X_allembszeros_large', embs_zero)

## Our toplogical features

In [5]:
PATH = 'D:/Datasets/CREMA-D/AudioWAV' ## PAth to dataset files

N_LAYERS = 12
N_HEADS = 12

features_traceh0 = []
features_non_attention = []
features_extended = []
features_l1 = []
features_linf = []

### Изменить в зависимости от структуры датасета. Приведено для CREMA-D
labels = []
text_to_label = {'ANG' : 0, 'HAP' : 1, 'DIS' : 2, 'FEA' : 3, 'NEU' : 4, 'SAD' : 5}

for filename in tqdm(os.listdir(PATH)):
    code = filename.split('_')[-2]
    labels.append(text_to_label[code])
    
    data, samplerate = sf.read(PATH + '/' + filename)
### ---------------------------------------------------------------------- 
    mfcc_feat = mfcc(data, samplerate)
  
    inputs = extractor(data, sampling_rate=samplerate, return_tensors="pt").input_values
    outp = model(inputs)
    
    res = outp.attentions
    embeds = outp.hidden_states
    
    bar_len = np.zeros(2 * N_LAYERS * N_HEADS)
    l1_len = np.zeros(N_LAYERS * N_HEADS)
    linf_len = np.zeros(N_LAYERS * N_HEADS) # better not use for now
    matrix_expanded_lens = np.zeros(3 * N_LAYERS * N_HEADS)
   
    n_len = res[0].shape[-1]
    for i in range(N_LAYERS):
        for j in range(N_HEADS):
            attention_map = res[i][0, j, :, :].detach()            
            symm_map = 1.0 - torch.maximum(attention_map, torch.transpose(attention_map, 0,1))
            symm_map -= torch.diag(torch.diag(symm_map))
            '''
            ########## BASIC FEATURES ##############################################################
            ### Средняя длина отрезков H0                                                          #
            bar_len[i * N_HEADS + j] = prim_tree(symm_map) / n_len                                 #
            ### Normalized trace of the __attention map__                                          #
            bar_len[N_HEADS * N_LAYERS + i * N_HEADS + j] = torch.mean(torch.diag(attention_map))  #
            '''
            ########## EXTENDED MATRIX FEATURES ####################################################
            ### Мера несимметричности - сумма верхнетреугольной части, делёггая на n^2             #
            matrix_expanded_lens[i * N_HEADS + j] = torch.mean(torch.triu(attention_map, 1))       #
            ### Average atttention to next token                                                   #
            matrix_expanded_lens[N_HEADS * N_LAYERS + i * N_HEADS + j] = torch.sum(torch.triu(attention_map, 1) - torch.triu(attention_map, 2)) / n_len
            ### Average attention to previous token                                                #
            matrix_expanded_lens[2 * N_HEADS * N_LAYERS + i * N_HEADS + j] = torch.sum(torch.tril(attention_map, -1) - torch.triu(attention_map, -2)) / n_len
            ########################################################################################
            
            ########## H0s over L_1-distance between rows ##########################################
            tmp = torch.cdist(attention_map, attention_map, p=1)                                   #
            l1_len[i * N_HEADS + j] = prim_tree(tmp) / n_len                                       #
            ########################################################################################
            
            ########## H0s over L_inf-distance between rows (lowest priority)#######################
            # Computational time is too long + this feature works very bad with our                #
            #                    traditional                 " H0 for attention "                  #
            #tmp = torch.cdist(attention_map, attention_map, p=np.inf)                             #
            #linf_len[i * N_HEADS + j] = prim_tree(tmp) / n_len                                    #
            ########################################################################################
    '''        
    #################### "Non-attentintion" FEATURES ###############################################
    ### Average H0 len betetween embeddings from each layer                                        #
    layer_h0s = np.zeros(N_LAYERS + 1)                                                             #
    for layer in range(N_LAYERS + 1):                                                              #
        layer_h0s[layer] = prim_tree(torch.cdist(embeds[layer], embeds[layer]))  / n_len           #
                                                                                                   #
    ### [approximated] RTD to very last layer and to initial embeddings                            #
    rtd_to_last = np.zeros(N_LAYERS)                                                               #
    rtd_to_start = np.zeros(N_LAYERS)                                                              #
    for layer in range(N_LAYERS):                                                                  #
            rtd_to_last[layer] = rtd_r(embeds[layer], embeds[-1])                                  #
            rtd_to_start[layer] = rtd_r(embeds[layer + 1], embeds[0])                              #                                                                   #
                                                                                                   #
    ### Average len H0 intervals between MFCC                                                      #
    mfcc_torch = torch.from_numpy(mfcc_feat)                                                       #
    h0 = prim_tree(torch.cdist(mfcc_torch, mfcc_torch))  / mfcc_feat.shape[0]                      #
    ################################################################################################
    '''
    features_traceh0.append(bar_len)
   # features_non_attention.append(np.hstack([np.mean(mfcc_feat, 0), layer_h0s, np.array([h0]), rtd_to_last, rtd_to_start]))
    features_extended.append(matrix_expanded_lens)
    features_l1.append(l1_len)
    features_linf.append(linf_len)

labels = np.array(labels)
for template in [features_traceh0, features_non_attention, features_extended, features_l1, features_linf]:
    template = np.vstack(template)

100%|████████████████████████████████████████████████████████████████████████████| 7442/7442 [6:41:21<00:00,  3.24s/it]


ValueError: need at least one array to concatenate

In [6]:
for template in [features_extended, features_l1, features_linf]:
    template = np.vstack(template)

In [7]:
np.save('X_basic_l1', features_l1)
np.save('X_basic_extended', features_extended)

In [14]:
np.save(SAVE_PATH + '/labels', labels)
np.save(SAVE_PATH + '/X_basic', features_traceh0)
np.save(SAVE_PATH + '/X_nonatt', features_non_attention)
np.save(SAVE_PATH + '/X_extended', features_extended)
np.save(SAVE_PATH + '/X_l1', features_l1)
np.save(SAVE_PATH + '/X_linf', features_linf)

In [8]:
def do_calc(X, y, seeds):
    print('Best C:', end = ' ')
    candidates = [0.05, 0.1, 0.2, 0.5, 0.8, 1, 2, 5] ### Candidates for parameter C
    q = np.zeros((seeds.shape[1], 3))
    for i in tqdm(range(seeds.shape[1])):
        X_train, X_2, y_train, y_2 = train_test_split(X, y, test_size=0.3, random_state=seeds[0, i])
        X_valid, X_test, y_valid, y_test = train_test_split(X_2, y_2, test_size=0.5, random_state=seeds[1, i])
    
        transit = StandardScaler()
        X_train = transit.fit_transform(X_train)
        X_valid = transit.transform(X_valid)
        X_test = transit.transform(X_test)
        
        ### Grid search for best C
        best_quality, best_c = 0.0, 1
        for cc in candidates:
            cls = LogisticRegression(penalty='l1', solver='liblinear', C=cc, max_iter=5000).fit(X_train, y_train)
            candidate_quality = cls.score(X_valid, y_valid)
            if candidate_quality > best_quality:
                best_quality = candidate_quality
                best_c = cc
        cls = LogisticRegression(penalty='l1', solver='liblinear', C=best_c, max_iter=10000).fit(X_train, y_train)
        print(best_c, end = ' ')
        q[i][0] = cls.score(X_train, y_train) * 100.0
        q[i][1] = cls.score(X_valid, y_valid) * 100.0
        q[i][2] = cls.score(X_test, y_test) * 100.0
    return q

In [13]:
np.random.seed(42)

N_RERUNS = 5
seeds = np.random.randint(1000, size=(2, N_RERUNS))

y = np.load(SAVE_PATH + '/labels.npy')
### Test 0 - the most basic features
X = np.load(SAVE_PATH + '/X_allembs_large.npy')
q0 = do_calc(X, y, seeds)
print('Mean est. quality :', np.mean(q0[:, -1]), '+-', np.std(q0[:, -1]))

X = np.load(SAVE_PATH + '/X_allembszeros_large.npy')
q1 = do_calc(X, y, seeds)
print('Zero est. quality :', np.mean(q1[:, -1]), '+-', np.std(q1[:, -1]))

Best C: 

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

0.1 

 20%|████████████████▌                                                                  | 1/5 [09:38<38:32, 578.10s/it]

0.1 

...

0.05 

 80%|██████████████████████████████████████████████████████████████████▍                | 4/5 [40:11<09:57, 597.71s/it]

0.1 

100%|███████████████████████████████████████████████████████████████████████████████████| 5/5 [50:03<00:00, 600.71s/it]

Zero est. quality : 72.82005371530886 +- 0.8746051739888873


In [ ]:
np.random.seed(42)

N_RERUNS = 5
seends = np.random.randint(1000, size=(2, N_RERUNS))

y = np.load(SAVE_PATH + '/labels.npy')
### Test 0 - the most basic features
X = np.load(SAVE_PATH + '/X_basic.npy')
q0 = do_calc(X, y, seeds)
print('Basic features est. quality :', np.mean(q0[:, -1]), '+-', np.std(q0[:, -1]))

X = np.load(SAVE_PATH + '/X_nonatt.npy')
q0 = do_calc(X, y, seeds)
print('Non-attention features est. quality :', np.mean(q0[:, -1]), '+-', np.std(q0[:, -1]))

X = np.load(SAVE_PATH + '/X_extended.npy')
q0 = do_calc(X, y, seeds)
print('Extended features est. quality :', np.mean(q0[:, -1]), '+-', np.std(q0[:, -1]))

X = np.load(SAVE_PATH + '/X_l1.npy')
q0 = do_calc(X, y, seeds)
print('L1 est. quality :', np.mean(q0[:, -1]), '+-', np.std(q0[:, -1]))

#X = np.load(SAVE_PATH + '/X_linf.npy') --- NO!
#q0 = do_calc(X, y, seeds)
#print('L_inf est. quality :', np.mean(q0[:, -1]), '+-', np.std(q0[:, -1]))

### Test 1 - basic features + non-attention
X = np.hstack([np.load(SAVE_PATH + '/X_basic.npy'),
              np.load(SAVE_PATH + '/X_nonatt.npy')])
q1 = do_calc(X, y, seeds)
print('Basic+non-attention features est. quality :', np.mean(q1[:, -1]), '+-', np.std(q1[:, -1]))

### Test 2 - full attention
X = np.hstack([np.load(SAVE_PATH + '/X_basic.npy'),
              np.load(SAVE_PATH + '/X_extended.npy')])
q2 = do_calc(X, y, seeds)
print('all attention features est. quality :', np.mean(q2[:, -1]), '+-', np.std(q2[:, -1]))

### Test 3 - l1
X = np.hstack([np.load(SAVE_PATH + '/X_basic.npy'),
              np.load(SAVE_PATH + '/X_l1.npy')])
q3 = do_calc(X, y, seeds)
print('basic + l1-features est. quality :', np.mean(q3[:, -1]), '+-', np.std(q3[:, -1]))

### Test 4 - l_inf --- NO!
#X = np.hstack([np.load(SAVE_PATH + '/X_basic.npy'),
#              np.load(SAVE_PATH + '/X_linf.npy')])
#q4 = do_calc(X, y, seeds)
#print('basic + linf-features est. quality :', np.mean(q4[:, -1]), '+-', np.std(q4[:, -1]))

### Test B - best for CREMA-D
X = np.hstack([np.load(SAVE_PATH + '/X_basic.npy'),
              np.load(SAVE_PATH + '/X_nonatt.npy'),
              np.load(SAVE_PATH + '/X_extended.npy'),
              np.load(SAVE_PATH + '/X_l1.npy')])
qB = do_calc(X, y, seeds)
print('Basic+non-attention+additional+l1 features est. quality :', np.mean(qB[:, -1]), '+-', np.std(qB[:, -1]))